### Imports

In [1]:
# %pip install pydirectory

In [2]:
import uuid
from bigdata_a3_utils import *
from pathlib import Path
import os
import tarfile
import pandas as pd
import shutil
import time, matplotlib.pyplot as plt, seaborn as sns, matplotlib.ticker as ticker
import numpy as np
from datasets import load_dataset, load_from_disk
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import json

### Files/folders

In [3]:
raw_files_path = "D:\\AS3\\Raw"
extraction_path = "D:\\AS3\\temp"

review_pkls_path = "C:\\Big Data\\A3\\Data\\review_pkl" #path of the review pkl files
meta_pkls_path = "C:\\Big Data\\A3\\Data\\meta_pkl"     #path of the meta pkl files

### Categories to process

In [4]:
categories = ['Unknown', 'Magazine_Subscriptions', 'Movies_and_TV', "Cell_Phones_and_Accessories", "Clothing_Shoes_and_Jewellery", "Digital_Music", "Hanmade_Products", "Baby_Products", "Beauty_and_Personal_Care", "Electronics"] # These are the ones that we have left to run

### Load Files to disk

In [5]:
# def load_compressed_dataset(compressed_path , extract_dir = None, cleanup_after_load: bool = True):
#     """
#     Load a dataset from a compressed archive (tar.gz, tar.bz2, or tar.xz).

#     Args:
#         compressed_path: Path to the compressed dataset file
#         extract_dir: Directory to extract files to (defaults to a temporary directory)
#         cleanup_after_load: Whether to delete the extracted files after loading
#                            (only applies to auto-generated temp directories)

#     Returns:
#         The loaded dataset (Dataset or DatasetDict)

#     Raises:
#         ValueError: If the file doesn't exist or isn't a supported compressed file
#     """
#     compressed_path = Path(compressed_path)

#     if not compressed_path.exists():
#         raise ValueError(f"File not found: {compressed_path}")

#     # check file extension
#     valid_extensions = [".tar.gz", ".tar.bz2", ".tar.xz"]
#     is_valid = False

#     for ext in valid_extensions:
#         if compressed_path.name.endswith(ext):
#             is_valid = True
#             break

#     if not is_valid:
#         raise ValueError(f"Expected a compressed tar file (.tar.gz, .tar.bz2, or .tar.xz), got: {compressed_path}")

#     # get the expected directory name (remove the extension)
#     dir_name = compressed_path.name
#     for ext in valid_extensions:
#         if dir_name.endswith(ext):
#             dir_name = dir_name[:-len(ext)]
#             break

#     # create extraction directory
#     is_temp_dir = extract_dir is None
#     if is_temp_dir:
#         extract_dir = compressed_path.parent / f"temp_{uuid.uuid4().hex}"
#     else:
#         extract_dir = Path(extract_dir)

#     extract_dir.mkdir(parents=True, exist_ok=True)

#     try:
#         # extract archive
#         print(f"Extracting {compressed_path} to {extract_dir}...")
#         with tarfile.open(compressed_path, "r:*") as tar:
#             tar.extractall(path=extract_dir)

#         # dataset should be in a subdirectory matching the original directory name
#         dataset_dir = extract_dir / dir_name

#         if not dataset_dir.exists():
#             # try to find any directory
#             extracted_folders = [f for f in extract_dir.iterdir() if f.is_dir()]
#             if not extracted_folders:
#                 raise ValueError(f"No folders found in extracted archive: {compressed_path}")
#             dataset_dir = extracted_folders[0]
#             print(f"Using extracted directory: {dataset_dir}")

#         # load dataset
#         print(f"Loading dataset from {dataset_dir}...")
#         dataset = load_from_disk(str(dataset_dir))

#         return dataset

#     finally:
#         # clean up only if it's a temporary directory we created AND cleanup is requested
#         if cleanup_after_load and is_temp_dir and extract_dir.exists():
#             print(f"Cleaning up temporary directory: {extract_dir}")
#             shutil.rmtree(extract_dir)

In [6]:
# This function handles the extraction of the brands
def extract_brand(details, store):
    try:
        if isinstance(details, dict) and "brand" in details and details["brand"]:
            return details["brand"]
    except Exception:
        pass
    if isinstance(store, str) and store.strip():
        return store
    return "Unknown"

### Clean categories

In [7]:
def merge_dfs(rev, meta, category):
    output_dir = r"D:\AS3\pkl_dir"
    print("Merging review and meta...")
    merged_df = pd.merge(
        rev,
        meta,
        left_on="parent_asin",
        right_on="parent_asin",
        how="inner"
        # suffixes=('_reviews', '_meta')
        )
    print("Merged")
    output_file = os.path.join(output_dir, f"{category}_cleaned_merged.pkl.bz2")
    merged_df.to_pickle(output_file, compression="bz2")
    return merged_df

In [8]:
# # this function merges the review and meta data dataframes, cleans them 
# # and returns the datframe made to ensure that it was put together and contains data
# def clean_data(category, merged_df):
#     output_dir = r"D:\AS3\pkl_dir"

#     print("Filtering invalid ratings...")
#     merged_df = merged_df[merged_df["rating_x"].between(1.0, 5.0, inclusive="both")]

#     print("Dropping empty review text...")
#     merged = merged_df[merged_df["text_x"].notna() & (merged_df["text_x"].str.strip() != "")]

#     print("Extracting brand from metadata...")
#     merged["brand"] = merged.apply(lambda row: extract_brand(row.get("details"), row.get("store")), axis=1)

#     print("Removing duplicate reviews...")
#     merged.drop_duplicates(subset=["user_id", "asin", "text_x"], keep="first", inplace=True)

#     print("Computing review length...")
#     merged["review_length"] = merged["text"].str.split().apply(len)

#     print("Extracting year from timestamp...")
#     merged["year"] = pd.to_datetime(merged["timestamp"], unit="ms", errors="coerce").dt.year

#     output_file = os.path.join(output_dir, f"{category}_cleaned_merged.pkl.bz2")
#     merged.to_pickle(output_file, compression="bz2")

#     print(" All cleaning steps completed.")
#     print(" DF shape.")
    
#     print(merged.shape)

In [9]:
# # this function merges the review and meta data dataframes, cleans them 
# # and returns the datframe made to ensure that it was put together and contains data
# def clean_data(category, merged_df):
#     # output_dir = r"D:\AS3\pkl_dir"

#     print("Filtering invalid ratings...")
#     merged_df = merged_df[merged_df["rating"].between(1.0, 5.0, inclusive="both")]

#     print("Dropping empty review text...")
#     if "text" in merged_df.columns:
#         merged_df = merged_df[merged_df["text"].notna() & (merged_df["text"].str.strip() != "")]

#     print("Extracting brand from metadata...")
#     if "brand" in merged_df.columns:
#         merged_df["brand"] = merged_df.apply(lambda row: extract_brand(row.get("details"), row.get("store")), axis=1)

#     print("Removing duplicate reviews...")
#     merged_df.drop_duplicates(subset=["user_id", "asin", "text"], keep="first", inplace=True)

#     print("Computing review length...")
#     merged_df["review_length"] = merged_df["text"].str.split().apply(len)

#     print("Extracting year from timestamp...")
#     merged_df["year"] = pd.to_datetime(merged_df["timestamp"], unit="ms", errors="coerce").dt.year

#     # output_file = os.path.join(output_dir, f"{category}_cleaned_merged.pkl.bz2")
#     # merged.to_pickle(output_file, compression="bz2")

#     print(" All cleaning steps completed.")
#     print(" DF shape.")
    
#     print(merged_df.shape)
#     return merged_df

In [ ]:
def clean_data(category, review_df, meta_df):

    output_dir = r"D:\AS3\pkl_dir"
    os.makedirs(output_dir, exist_ok=True)

    print("=== Merging review and meta on 'parent_asin' (LEFT JOIN) ===")
    merged = pd.merge(review_df, meta_df, on="parent_asin", how="left")
    print(f"After merge: {len(merged)} rows")

    # invalid ratings
    if "rating" in merged.columns:
        before = len(merged)
        merged = merged[merged["rating"].between(1.0, 5.0, inclusive="both")]
        print(f"After filtering invalid ratings: {len(merged)} rows (dropped {before - len(merged)})")
    
    # empty review texts
    if "text" in merged.columns:
        before = len(merged)
        merged = merged[merged["text"].notna() & (merged["text"].str.strip() != "")]
        print(f"After dropping empty text: {len(merged)} rows (dropped {before - len(merged)})")

    #  brand
    print("Extracting brand from metadata...")
    merged["brand"] = merged.apply(
        lambda row: extract_brand(row.get("details"), row.get("store")), axis=1
    )
    merged["brand"].fillna("Unknown", inplace=True)

    # duplicates
    before = len(merged)
    merged.drop_duplicates(subset=["user_id", "asin", "text"], keep="first", inplace=True)
    print(f"After removing duplicates: {len(merged)} rows (dropped {before - len(merged)})")

    # derived columns
    if "text" in merged.columns:
        merged["review_length"] = merged["text"].str.split().apply(len)

    if "timestamp" in merged.columns:
        merged["year"] = pd.to_datetime(merged["timestamp"], unit="ms", errors="coerce").dt.year

    # cleaned data
    output_file = os.path.join(output_dir, f"{category}_cleaned_merged.pkl.bz2")
    merged.to_pickle(output_file, compression="bz2")
    print(f"Saved cleaned file to {output_file}")

    return merged

### Process Categories

In [11]:
def load_raw_rev_and_meta(extraction_path, category):
    file_path = os.path.join(raw_files_path, f"raw_review_{category}.tar.bz2")
    print(file_path)
    review_dataset = load_compressed_dataset(file_path, extraction_path)
    review_df = review_dataset["full"].to_pandas() 

    file_path = os.path.join(raw_files_path, f"raw_meta_{category}.tar.bz2")
    print(file_path)
    meta_dataset = load_compressed_dataset(file_path, extraction_path)
    meta_df = review_dataset["full"].to_pandas()

    return review_df, meta_df


In [ ]:
# Ensure the function iterates over files in the directory and constructs full paths
categories = ["All_beauty", "Appliances"]
# categories = ["All_Beauty"]

for category in categories:
   rev, meta =  load_raw_rev_and_meta(extraction_path, category)
   # merged_df = merge_dfs(rev, meta)
   # print(merged_df)
   clean_data(category, rev, meta)

D:\AS3\Raw\raw_review_All_Beauty.tar.bz2
Extracting D:\AS3\Raw\raw_review_All_Beauty.tar.bz2 to D:\AS3\temp...
Loading dataset from D:\AS3\temp\raw_review_All_Beauty...
D:\AS3\Raw\raw_meta_All_Beauty.tar.bz2
Extracting D:\AS3\Raw\raw_meta_All_Beauty.tar.bz2 to D:\AS3\temp...
Loading dataset from D:\AS3\temp\raw_meta_All_Beauty...
=== Merging review and meta on 'parent_asin' (LEFT JOIN) ===
After merge: 75797082 rows
Extracting brand from metadata...


KeyError: Index(['user_id', 'asin', 'text'], dtype='object')

In [ ]:
# # For loop to iterate the categories, clean them and convert to compressed pkl zips
# # also removes the uncompressed files from the system once they've been done
# for category in categories:
#     base_path = r"" # replace with path to tar files
#     meta_path = r"" # replace with path to meta pkl files
#     review_path = r"" # replace with path to review pkl files

#     # review pkled folder
#     rev_pkl  = r"/root/Data/output_folder musical-video_games/reviews_pkl" # Make sure this is the folder with review .pkl batches
#     meta_pkl = r"/root/Data/output_folder musical-video_games/meta_pkl"  # Make sure this is the folder with meta .pkl batches

    
#     load_compressed_dataset()

#     review_df = convert_to_df(review_path, category)
#     meta_df = convert_to_df(meta_path, category)
#     cleaned = clean_data(category, review_df, meta_df)
#     print(cleaned)
#     del cleaned
#     del meta_df
#     del review_df

#     # remove the review and meta pkl files that aren't compressed
#     if os.path.exists(rev_pkl):
#         shutil.rmtree(rev_pkl)
#     else:
#         print(f"{rev_pkl} path does not exist")

#     if os.path.exists(meta_pkl):
#         shutil.rmtree(meta_pkl)
#     else:
#         print(f"{meta_pkl} path does not exist")

In [ ]:
# import os
# import tarfile
# import json

# def extract_tar_bz2(tar_path, extract_to):
#     with tarfile.open(tar_path, 'r:bz2') as tar:
#         tar.extractall(path=extract_to)

# def process_jsonl_file(file_path, output_path, clean_fn=None):
#     with open(file_path, 'r') as infile, open(output_path, 'w') as outfile:
#         for line in infile:
#             try:
#                 obj = json.loads(line)
#                 if clean_fn:
#                     obj = clean_fn(obj)
#                     if obj is None:
#                         continue
#                 outfile.write(json.dumps(obj) + '\n')
#             except json.JSONDecodeError:
#                 continue

# def basic_clean_review(obj):
#     # Example: skip entries without reviewText
#     if not obj.get("reviewText"):
#         return None
#     return obj

# def basic_clean_meta(obj):
#     # Example: skip entries with no title
#     if not obj.get("title"):
#         return None
#     return obj

# def process_category(extraction_path, category):
#     rev_tar = os.path.join(raw_files_path, f"raw_review_{category}.tar.bz2")
#     meta_tar = os.path.join(raw_files_path, f"raw_meta_{category}.tar.bz2")

#     rev_dir = os.path.join(extraction_path, f"{category}_reviews")
#     meta_dir = os.path.join(extraction_path, f"{category}_meta")
#     os.makedirs(rev_dir, exist_ok=True)
#     os.makedirs(meta_dir, exist_ok=True)

#     extract_tar_bz2(rev_tar, rev_dir)
#     extract_tar_bz2(meta_tar, meta_dir)

#     # Assume file inside tar is named {category}.json.gz or json
#     review_path = [f for f in os.listdir(rev_dir) if f.endswith('.json') or f.endswith('.jsonl')][0]
#     meta_path = [f for f in os.listdir(meta_dir) if f.endswith('.json') or f.endswith('.jsonl')][0]

#     cleaned_review_out = os.path.join(extraction_path, f"{category}_cleaned_reviews.jsonl")
#     cleaned_meta_out = os.path.join(extraction_path, f"{category}_cleaned_meta.jsonl")

#     process_jsonl_file(os.path.join(rev_dir, review_path), cleaned_review_out, clean_fn=basic_clean_review)
#     process_jsonl_file(os.path.join(meta_dir, meta_path), cleaned_meta_out, clean_fn=basic_clean_meta)

#     print(f"✅ Cleaned files saved for {category}")

# # Example usage
# for category in ["All_beauty", "Appliances"]:
#     process_category(extraction_path, category)
